In [1]:
#import package
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
#前處理
#不採用前三個欄位
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
x1 = train[['CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']]
#類別型轉數值型
x1 = pd.get_dummies(x1)
x2 = train['Exited']

y = test[['CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']]
y = pd.get_dummies(y)

#標準化資料
scaler = StandardScaler()
scaler.fit(x1)
x_train_std = scaler.transform(x1)
#套模型
clf = RandomForestClassifier()
clf.fit(x_train_std,x2)
exited = clf.predict(y)

#create csv
df = pd.DataFrame({'RowNumber': test['RowNumber'], 'Exited': list(exited)})
df.to_csv(r'./predict_1.csv',columns=['RowNumber','Exited'],index=True,sep=',')

In [9]:
#import package
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

x1 = train[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'Tenure', 'NumOfProducts',  'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
x1 = pd.get_dummies(x1)
x2 = train['Exited']

#自行切割訓練資料
x1_train, x1_test, x2_train, x2_test = train_test_split(x1,x2, train_size=0.8,random_state=3)

#調整n_estimators
clf = RandomForestClassifier(n_estimators=250)
scaler = StandardScaler()
scaler.fit(x1_train)
x1_train_std = scaler.transform(x1_train)
x1_test_std = scaler.transform(x1_test)

clf.fit(x1_train_std, x2_train)

x1_test_predict = clf.predict(x1_test_std)
# test accuracy score
np.mean(x1_test_predict==x2_test)

y = test[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'Tenure', 'NumOfProducts',  'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
y = pd.get_dummies(y)

scaler.fit(x1)
x1_std = scaler.transform(x1)

clf.fit(x1_std, x2)

y_std = scaler.transform(y)
y_predict = clf.predict(y_std)
y_predict

df = pd.DataFrame({'RowNumber': test['RowNumber'], 'Exited': list(y_predict)})
df.to_csv(r'./predict_22.csv',columns=['RowNumber','Exited'],index=True,sep=',')

In [5]:
#import package
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

x1 = train[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'Tenure', 'NumOfProducts',  'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
x1 = pd.get_dummies(x1)
x2 = train['Exited']

y = test[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'Tenure', 'NumOfProducts',  'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
y = pd.get_dummies(y)

#沒有標準化的準確率較低
clf = RandomForestClassifier()
clf.fit(x1, x2)
y_predict = clf.predict(y)
y_predict

df = pd.DataFrame({'RowNumber': test['RowNumber'], 'Exited': list(y_predict)})
df.to_csv(r'./predict_3.csv',columns=['RowNumber','Exited'],index=True,sep=',')

In [10]:
#import package
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#train
#load data
data=pd.read_csv("train.csv")
X=data.iloc[:,0:-1]
y=data.iloc[:,-1]
train_X,test_X,train_y,test_y=train_test_split(X,y,train_size=0.8,stratify=y,random_state=3)

#另一種前處理方法
#標準化資料
sc=StandardScaler()
ad_train_X=train_X.iloc[:,[3,6,7,8,9,12]]
ad_test_X=test_X.iloc[:,[3,6,7,8,9,12]]
ad1_train_X=sc.fit_transform(ad_train_X)
ad1_test_X=sc.transform(ad_test_X)

#重新編碼
ohe=OneHotEncoder()
ad2_train_X=train_X.iloc[:,[4,5,10,11]]
ad2_test_X=test_X.iloc[:,[4,5,10,11]]
ad3_train_X=ohe.fit_transform(ad2_train_X).toarray()
ad3_test_X=ohe.transform(ad2_test_X).toarray()

df1=pd.DataFrame(ad1_train_X)
df2=pd.DataFrame(ad3_train_X)
for i in range(6,15):
    df1[i]=df2[i-6]
df3=pd.DataFrame(ad1_test_X)
df4=pd.DataFrame(ad3_test_X)
for i in range(6,15):
    df3[i]=df4[i-6]

#去掉不需要的欄位
df1.drop(columns=[8,9,11,12,13],inplace=True)
df3.drop(columns=[8,9,11,12,13],inplace=True)

adj_train_X=df1
adj_test_X=df3

#套用模型
clf=clf = RandomForestClassifier(n_estimators=300)
clf.fit(adj_train_X,train_y)
pred_y=clf.predict(adj_test_X)

#預測結果
accuracy=accuracy_score(test_y,pred_y)
precision=precision_score(test_y,pred_y,zero_division=0)
recall=recall_score(test_y,pred_y)
f1 = f1_score(test_y,pred_y)
final=0.3*accuracy+0.3*precision+0.4*f1
print("%.5f %.5f %.5f %.5f %.5f" %(final,accuracy,precision,recall,f1))
# print('訓練集: ',clf.score(adj_train_X,train_y))
# print('測試集: ',clf.scoreadj_test_X,test_y))


0.73174 0.86687 0.76777 0.49693 0.60335
